In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
import numpy as np

In [2]:
# Load the pre-trained model and tokenizer for English-Vietnamese translation
model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
embedding_model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [3]:
def cross_language_search_vi_en(query, documents):
    """Performs cross-language search (Vietnamese to English).

    Args:
        query: The Vietnamese search query.
        documents: A list of English documents to search.

    Returns:
        A list of documents, sorted by relevance to the query.
    """

    # Translate the query to English
    input_ids = tokenizer.encode(query, return_tensors="pt", padding=True, truncation=True)
    translated_ids = model.generate(input_ids)
    translated_query = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

    best_document = search_engine(translated_query, documents)

    return best_document


def search_engine(translated_query, documents):
    # Compute embeddings for the query and documents (using LaBSE for multilingual support)
    query_embedding = embedding_model.encode(translated_query)  # Use SentenceTransformer for embedding
    document_embeddings = embedding_model.encode(documents)     # Use SentenceTransformer for embedding

    # Calculate cosine similarity between the query and documents
    similarities = util.cos_sim(query_embedding, document_embeddings).flatten()

    # Get the index of the most similar document
    best_index = np.argmax(similarities.cpu().numpy())

    # Get the best matching document
    best_document = [documents[best_index]]

    return best_document


documents = [
    "A car is a widely used means of road transportation, essential in daily life for many people. Cars offer convenience, comfort, and safety for travel. With various types such as sedans, SUVs, hatchbacks, and crossovers, cars meet the diverse needs of users, from families to individuals. They not only make commuting easier but also contribute to one's lifestyle and personal expression.",
    "An airplane is an aerial vehicle that enables travel over vast distances in a short amount of time. Commercial airplanes carry hundreds of passengers on international flights, providing speed and convenience for long journeys. Private jets cater to businesspeople and celebrities, while helicopters are versatile for rescue missions and military operations. Airplanes not only offer superior mobility but also play crucial roles in numerous sectors, including tourism, cargo transport, and emergency services.",
    "A boat is a watercraft used for traveling on rivers, lakes, and oceans. With various types like sailboats, motorboats, yachts, and fishing vessels, boats provide enjoyable and relaxing experiences on the water. Sailboats glide gracefully using wind power, motorboats are powerful with modern engines, yachts are luxurious with all amenities, and fishing boats serve the fishing industry. Boats are not only used for recreational activities but also hold significant roles in various industries and services.",
    "Hola mundo",
    "こにちは"
]



In [4]:
search_engine("Sea shell at the seashore ", documents)

['A boat is a watercraft used for traveling on rivers, lakes, and oceans. With various types like sailboats, motorboats, yachts, and fishing vessels, boats provide enjoyable and relaxing experiences on the water. Sailboats glide gracefully using wind power, motorboats are powerful with modern engines, yachts are luxurious with all amenities, and fishing boats serve the fishing industry. Boats are not only used for recreational activities but also hold significant roles in various industries and services.']

In [5]:
# Example usage
query = "Chiếc thuyền ngoài khơi"  # Vietnamese search query
results = cross_language_search_vi_en(query, documents)

print("\nQuery:", query, "\n")
print("Search results:")
results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Query: Chiếc thuyền ngoài khơi 

Search results:


['A boat is a watercraft used for traveling on rivers, lakes, and oceans. With various types like sailboats, motorboats, yachts, and fishing vessels, boats provide enjoyable and relaxing experiences on the water. Sailboats glide gracefully using wind power, motorboats are powerful with modern engines, yachts are luxurious with all amenities, and fishing boats serve the fishing industry. Boats are not only used for recreational activities but also hold significant roles in various industries and services.']